In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd
import matplotlib.pyplot as plt
from os.path import expanduser, join, isfile

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import article_analysis.parse as aap

%load_ext autoreload
%autoreload 1
%aimport article_analysis.parse
%matplotlib inline

In [2]:
df = pd.read_json(expanduser('~/data/jstor/amj_raw/json_amj.txt'), lines=True)
df.shape

(5694, 13)

In [3]:
df.head(2)

,authors,data,discipline,doi,journaltitle,number,pagecount,pagerange,publication-date,reviewed-work,title,type,volume
0,John B. Miner,{'ocr': ['Communications Exodus-The study also...,"[Business & Economics Collection, Management &...",10.2307/254807,The Academy of Management Journal,1,2,127-128,1972-03-01T00:00:00Z,NaN,Success in Management Consulting and the Conce...,research-article,15
1,Charles R. McClure,{'ocr': ['The author's own recommendations are...,"[Business & Economics Collection, Management &...",10.2307/257446,The Academy of Management Review,2,3,312-314,1980-04-01T00:00:00Z,<product><source>\n <source>Information Strate...,,book-review,5


In [4]:
# df['doi'].to_csv('/Users/belikov/data/jstor/doi.csv.gz', compression='gzip')

In [5]:
corpus = dict(zip(df['doi'], [x['ocr'] for x in df['data']]))

In [6]:
len(corpus)

5694

In [7]:
fpath = expanduser('~/data/jstor/ngrams_new2')
destpath = fpath
suffix = 'pgz'
prefix = 'ngrams_corpus'

In [8]:
print('len corpus', len(corpus))
ss, max_index = aap.get_present_keys(fpath, prefix, suffix)
keys_to_process = set(corpus.keys()) - ss
print('keys to process len', len(keys_to_process))
remaining_corpus = {k: corpus[k] for k in keys_to_process}
print('remaning corpus len', len(remaining_corpus))
corpus_split = aap.split_corpus(remaining_corpus, 200)

len corpus 5694
keys to process len 5694
remaning corpus len 5694


In [9]:
lmtzr = WordNetLemmatizer()

In [ ]:
for j, chunk in zip(range(len(corpus_split)), corpus_split):
    corpus_ngrams = {}
    for doi, article in chunk.items():
        print('{0} '.format(doi), end='')
        article_phrases = aap.transform_article(article)
        print('{1} '.format(doi, len(article_phrases)), end='')
        ngram_dict = aap.compute_ngrams(article_phrases)
        orders = sorted(ngram_dict.keys())
        for o in orders:
            ngram_dict[o] = aap.transform_counter(ngram_dict[o], 'lemma', lmtzr)
        corpus_ngrams[doi] = ngram_dict
    with gzip.open(join(destpath, 'ngrams_corpus_{0}.pgz'.format(j + 1 + max_index )), 'wb') as fp:
        pickle.dump(corpus_ngrams, fp)
    print('\n{0} chunks, {1:.2f} % complete'.format(j, 100*(j)/len(corpus_split)))

In [11]:
# doi_ = '10.2307/3069361'
# doi_ = '10.2307/254807'
# doi_ = '10.2307/2667048'
# doi_ = '10.2307/254961'
# doi_ = '10.1525/sop.2008.51.1.163'
# mask = (df['doi'] == doi_)
# article = df.loc[mask, 'data'].values[0]['ocr']
# # ar_phrases = aap.transform_article(article, verbose=True)
# ar_phrases = aap.transform_article(article)
# ng_dict = aap.compute_ngrams(ar_phrases)
# ng_dict[2]